# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [ ]:
# try:
#   import google.colab
#   IN_COLAB = True
# except:
#   IN_COLAB = False

In [ ]:
# if IN_COLAB:
#     !apt-get install openjdk-8-jdk-headless -qq > /dev/null
#     !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
#     !tar xf spark-3.3.2-bin-hadoop3.tgz
#     !mv spark-3.3.2-bin-hadoop3 spark
#     !pip install -q findspark
#     import os
#     os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#     os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark_url = 'local'

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Assignment')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()
sc = spark.sparkContext

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [5]:
path = 'netflix-rotten-tomatoes-metacritic-imdb.csv'
df = spark.read.csv(path,header=True,inferSchema=True)
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- Series or Movie: string (nullable = true)
 |-- Hidden Gem Score: double (nullable = true)
 |-- Country Availability: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writer: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- View Rating: string (nullable = true)
 |-- IMDb Score: string (nullable = true)
 |-- Rotten Tomatoes Score: string (nullable = true)
 |-- Metacritic Score: string (nullable = true)
 |-- Awards Received: double (nullable = true)
 |-- Awards Nominated For: double (nullable = true)
 |-- Boxoffice: string (nullable = true)
 |-- Release Date: string (nullable = true)
 |-- Netflix Release Date: string (nullable = true)
 |-- Production House: string (nullable = true)
 |-- Netflix Link: string (nullable = true)
 |-- IMDb Link: string (null

## What is the maximum and average of the overall hidden gem score?

In [7]:
from pyspark.sql.functions import avg, max
df.select(max('Hidden Gem Score'),avg('Hidden Gem Score')).show()

+---------------------+---------------------+
|max(Hidden Gem Score)|avg(Hidden Gem Score)|
+---------------------+---------------------+
|                  9.8|    5.937551386501226|
+---------------------+---------------------+



## How many movies that are available in Korea?

In [26]:
df.filter(df['Languages'].contains("Korea")).count()

735

## Which director has the highest average hidden gem score?

In [17]:
from pyspark.sql.functions import col

df.groupBy('Director') \
    .agg(avg('Hidden Gem Score').alias('avg_hidden_gem_score')) \
    .orderBy(col('avg_hidden_gem_score').desc()) \
    .show(1)

+-----------+--------------------+
|   Director|avg_hidden_gem_score|
+-----------+--------------------+
|Dorin Marcu|                 9.8|
+-----------+--------------------+
only showing top 1 row



## How many genres are there in the dataset?

In [25]:
from pyspark.sql.functions import explode, split

df.withColumn('Genre', explode(split('Genre', ', '))) \
    .select('Genre') \
    .distinct() \
    .count()

28